In traditional robotic systems, joint torque sensors are used to measure the total torque at a joint, including both internal and external components. However, due to the inherent properties of HDTs, such as high friction and complex mechanical behavior, a significant portion of the measured torque does not contribute to the actual movement of the robot's link but is rather dissipated internally.

To address this, the current research paradigm emphasizes the importance of isolating and accurately estimating the external torque component. External torques are critical as they directly affect the robot's interaction with its environment, such as during tasks involving physical manipulation or contact with objects. While the internal torques due to friction and position-dependent effects are known and can be compensated for, the external torque estimation remains challenging and is the focus of this work.

This thesis explores various methods to estimate these external torques, leveraging both traditional analytical approaches and modern machine learning techniques. The analytical methods provide a foundation by modeling known internal forces, while the machine learning approaches aim to capture the residuals and complexities not accounted for by these models, particularly the external torques. This dual approach not only enhances the accuracy of torque estimation but also contributes to the development of a more comprehensive virtual joint torque sensor.

\ref{fig:somelabel} is the image

\begin{equation}
\int f(x)\,\text{d}x = F(x)
\label{eq1}
\tag{eq1text}
\end{equation}

\ref{eq1} is the equation

In [32]:

import numpy as np

def dh_transform(a, alpha, d, theta):
    """
    Compute Denavit-Hartenberg transformation matrix.
    """
    return np.array([
        [np.cos(theta), -np.sin(theta) * np.cos(alpha),  np.sin(theta) * np.sin(alpha), a * np.cos(theta)],
        [np.sin(theta),  np.cos(theta) * np.cos(alpha), -np.cos(theta) * np.sin(alpha), a * np.sin(theta)],
        [0,             np.sin(alpha),                 np.cos(alpha),                 d],
        [0,             0,                             0,                             1]
    ])

def compute_geometric_jacobian(dh_params, joint_angles):
    """
    Compute the geometric Jacobian for a robot given DH parameters and joint angles.

    :param dh_params: List of DH parameters [(a, alpha, d, theta), ...]
    :param joint_angles: List of joint angles [theta1, theta2, ..., thetaN]
    :return: Geometric Jacobian matrix (6 x N)
    """
    num_joints = len(joint_angles)
    T = np.eye(4)
    transformations = [T.copy()]
    
    # Compute forward kinematics to get transformations
    for i, (a, alpha, d, theta) in enumerate(dh_params):
        theta += joint_angles[i]  # Update theta with joint angle
        T_i = dh_transform(a, alpha, d, theta)
        T = np.dot(T, T_i)
        transformations.append(T.copy())
    
    # Initialize Jacobian matrix
    jacobian = np.zeros((6, num_joints))
    
    # End-effector position
    T_0_n = transformations[-1]
    o_n = T_0_n[:3, 3]

    # Compute each column of the Jacobian
    for i in range(num_joints):
        T_0_i = transformations[i]
        o_i = T_0_i[:3, 3]
        z_i = T_0_i[:3, 2]

        jacobian[:3, i] = np.cross(z_i, (o_n - o_i))  # Linear part
        jacobian[3:, i] = z_i  # Angular part

    return jacobian

# Updated DH parameters based on the provided table
dh_params = [
    (0, -np.pi/2, 0.26, 0),           # Link 1
    (0.48, 0, 0, -np.pi/2),       # Link 2
    (0, np.pi/2, 0, np.pi/2),        # Link 3
    (0, -np.pi/2, 0.7, 0),          # Link 4
    (0, np.pi/2, 0, 0),              # Link 5
    (0, 0, 0.1735, 0)                 # Link 6
]

# Example joint angles
joint_angles = [0, 0, np.pi/3, 0, 0, 0]

# Compute the Jacobian
jacobian = compute_geometric_jacobian(dh_params, joint_angles)
print("Geometric Jacobian:")
print(np.round(jacobian, 3))

jacobian.T@np.array([1,0,1,0,1,0])

Geometric Jacobian:
[[ 0.     0.917  0.437  0.     0.087  0.   ]
 [ 0.756  0.     0.    -0.     0.    -0.   ]
 [-0.    -0.756 -0.756  0.    -0.15   0.   ]
 [ 0.     0.     0.     0.866  0.     0.866]
 [ 0.     1.     1.     0.     1.     0.   ]
 [ 1.     0.     0.     0.5    0.     0.5  ]]


array([2.64829870e-18, 1.16027681e+00, 6.80276810e-01, 3.06161700e-17,
       9.36494592e-01, 3.06161700e-17])

#### Introduce motor currents as a method of estimating joint torque